In [2]:
import pandas as pd
from sqlalchemy import create_engine
print('Done')
engine = create_engine('sqlite:///ventas.db')

Done


In [13]:
# Definir tamaño del bloque
chunksize = 500_000

# Cargar por partes y subir a la tabla 'sales'
for i, chunk in enumerate(pd.read_csv(
    'sales.csv',
    chunksize=chunksize,
    dtype={
        'promo_bin_1': str,
        'promo_bin_2': str,
        'promo_discount_type_2': str
    }
)):
    print(f"Subiendo bloque {i + 1}")
    chunk.to_sql('sales', con=engine, if_exists='append', index=False)

print("Carga completa ✅")

Subiendo bloque 1
Subiendo bloque 2
Subiendo bloque 3
Subiendo bloque 4
Subiendo bloque 5
Subiendo bloque 6
Subiendo bloque 7
Subiendo bloque 8
Subiendo bloque 9
Subiendo bloque 10
Subiendo bloque 11
Subiendo bloque 12
Subiendo bloque 13
Subiendo bloque 14
Subiendo bloque 15
Subiendo bloque 16
Subiendo bloque 17
Subiendo bloque 18
Subiendo bloque 19
Subiendo bloque 20
Subiendo bloque 21
Subiendo bloque 22
Subiendo bloque 23
Subiendo bloque 24
Subiendo bloque 25
Subiendo bloque 26
Subiendo bloque 27
Subiendo bloque 28
Subiendo bloque 29
Subiendo bloque 30
Subiendo bloque 31
Subiendo bloque 32
Subiendo bloque 33
Subiendo bloque 34
Subiendo bloque 35
Subiendo bloque 36
Subiendo bloque 37
Subiendo bloque 38
Subiendo bloque 39
Carga completa ✅


In [14]:
# Cargar los CSV
# Lectura por chunks
df_products = pd.read_csv('product_hierarchy.csv')
df_stores = pd.read_csv('store_cities.csv')

# Subir los DataFrames como tablas
df_products.to_sql('product_hierarchy', engine, if_exists='replace', index=False)
df_stores.to_sql('store_cities', engine, if_exists='replace', index=False)


144

In [15]:
from sqlalchemy import inspect

inspector = inspect(engine)
inspector.get_table_names()


['product_hierarchy', 'sales', 'store_cities']

In [16]:
query = "SELECT COUNT(*) FROM sales"
pd.read_sql_query(query, engine)


,COUNT(*)
0,19454838


In [18]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("ALTER TABLE sales ADD COLUMN anio TEXT;"))
    conn.execute(text("ALTER TABLE sales ADD COLUMN mes TEXT;"))
    conn.execute(text("UPDATE sales SET anio = strftime('%Y', date);"))
    conn.execute(text("UPDATE sales SET mes = strftime('%m', date);"))



In [21]:
query = """
SELECT 
    product_id, 
    SUM(sales) AS ventas_por_mes
FROM sales
WHERE anio = '2018'
GROUP BY product_id
ORDER BY ventas_por_mes DESC
LIMIT 10
"""

df_ventas_2018 = pd.read_sql_query(query, engine)
df_ventas_2018


,product_id,ventas_por_mes
0,P0103,253359.0
1,P0438,249535.0
2,P0388,116284.0
3,P0364,95373.0
4,P0051,84698.0
5,P0131,73746.0
6,P0569,68329.0
7,P0333,64459.0
8,P0590,56797.0
9,P0500,48189.0


In [24]:
query = """ 
WITH ventas_2018 AS (
    SELECT
        mes, 
        product_id, 
        SUM(sales) AS total_ventas
    FROM sales
    WHERE anio = '2018'
    GROUP BY mes, product_id
),
max_ventas AS (
    SELECT
        mes, 
        MAX(total_ventas) AS max_ventas
    FROM ventas_2018
    GROUP BY mes
)
SELECT v.*
FROM ventas_2018 v
JOIN max_ventas m
  ON v.mes = m.mes AND v.total_ventas = m.max_ventas
ORDER BY CAST(v.mes AS INTEGER)
"""

df_top_2018 = pd.read_sql_query(query, engine)
df_top_2018


,mes,product_id,total_ventas
0,01,P0438,21326.0
1,02,P0438,21485.0
2,03,P0438,23154.0
3,04,P0438,24392.0
4,05,P0438,23780.0
5,06,P0103,26307.0
6,07,P0103,27884.0
7,08,P0103,29185.0
8,09,P0438,32882.0
9,10,P0438,21267.0



### Necesito saber en cuántas tiendas se implementaron promociones del tipo 1 en el canal 1 durante el año 2019. ¿Puedes darme ese dato?

In [28]:
query = """
WITH temp_table AS (
    SELECT store_id
    FROM sales
    WHERE anio = "2019"
    AND promo_bin_1 != ''
)

SELECT COUNT(DISTINCT store_id) AS total_tiendas
FROM temp_table
"""

result = pd.read_sql_query(query, engine)
result

,total_tiendas
0,144


### Me gustaría saber cuál fue la ciudad que tuvo el mayor volumen de ventas en general durante todo el periodo de 2017 a 2019. ¿Tienes esa información disponible?

In [31]:
query = """
    WITH table_join AS (
        SELECT s.store_id, s.sales, st.city_id
        FROM sales AS s
        LEFT JOIN store_cities AS st
        ON s.store_id = st.store_id    
    ),

    table_temp AS (
        SELECT city_id, SUM(sales) AS total_sales
        FROM table_join
        GROUP BY city_id 
    )

    SELECT *
    FROM table_temp
    ORDER BY total_sales DESC

"""

result = pd.read_sql_query(query, engine)
result

,city_id,total_sales
0,C014,2573439.552
1,C022,906317.927
2,C031,803258.172
3,C036,375269.861
4,C024,286316.087
5,C020,270745.066
6,C025,224364.953
7,C004,203208.971
8,C006,184889.866
9,C029,180286.465


### ¿Podrías decirme cuál fue el promedio de stock disponible en todas las tiendas durante el año 2017? Estoy interesado en conocer ese dato.

In [34]:
query = """
    SELECT AVG(stock) as avg_stock_2017
    FROM sales
    WHERE anio = '2017'
"""
result = pd.read_sql_query(query, engine)
result

,avg_stock_2017
0,16.615351


### Estoy buscando saber cuántas veces se aplicaron descuentos en el canal 2 durante el mes de diciembre de 2018. ¿Puedes ayudarme con esa información?

Como jefe de analítica, tengo algunas preguntas realizadas por negocio que necesito que respondas utilizando las herramientas de Pandas, PySpark y SQL. A continuación, te presento las preguntas junto con los detalles requeridos para cada una:




¿Cuál fue el producto con el precio de venta más alto en cada tienda durante el tercer trimestre de 2019? Me gustaría conocer esos detalles.
Estoy interesado en saber cuántas tiendas experimentaron un aumento de ventas del 20% o más en comparación con el año anterior. ¿Tienes esos datos disponibles?
Quisiera saber cuál fue el producto que experimentó la mayor variación de precio a lo largo del periodo de 2017 a 2019. ¿Puedes proporcionarme esa información?
¿Sabes en qué mes y año se registró la mayor cantidad de ingresos generados en todas las tiendas? Estoy interesado en conocer ese dato importante.
Me gustaría saber cuál fue el tamaño promedio de las tiendas en cada ciudad durante el año 2018. ¿Puedes darme esa información?